In [ ]:
import pandas as pd
import re
import numpy as np
import os
import glob

In [ ]:
import configparser

config = configparser.RawConfigParser()
config.read('parametros.config')

PARAMETROS = dict(config.items('Padronizacao_conversao'))
PARAMETROS_GERAIS = dict(config.items('Geral'))

In [ ]:
regex = {}
regex['ATIVO']                          = {'regex':  "1            "}
regex['ATIVO_CIRCULANTE']               = {'regex':  "12           "}
regex['CREDITO_OPER_PS']                = {'regex':  "123          "}
regex['PASSIVO']                        = {'regex':  "2            "}
regex['PASSIVO_CIRCULANTE']             = {'regex':  "21           "}
regex['PASSIVO_NAO_CIRCULANTE']         = {'regex':  "23           "}
regex['PL']                             = {'regex':  "25           "}
regex['RECEITA_COM_ASSISTENCIA_A_SAUDE']= {'regex':  "31           "}
regex['CONTRAPRESTACAO_PRE']            = {'regex':  "311[^7].1       "}
regex['CONTRAPRESTACAO_POS']            = {'regex':  "311[^7].2       "}
regex['CORRESP_CEDIDA_PRE']             = {'regex':  "0",
                                              '2018':"3117.1       ",
                                              '2019':"3117.1       "}
regex['CORRESP_CEDIDA_POS']             = {'regex':  "0",
                                              '2018':"3117.2       ",
                                              '2019':"3117.2       "}
regex['EVENTOS_INDENIZAVEIS_LIQUIDOS']  = {'regex':  "41           "}
regex['EVENTOS_INDENIZAVEIS_PRE']       = {'regex':  "411..1       "}
regex['EVENTOS_INDENIZAVEIS_POS']       = {'regex':  "411..2       "}
regex['VARIACAO_PEONA']                 = {'regex':  "414          "}
regex['DESPESA_COM']                    = {'regex':  "43           "}
regex['DESPESA_ADM']                    = {'regex':  "46           "}
regex['PROVISOES_TECNICAS']             = {'regex' : "2[1|3]1          "}
regex['PPCNG']                          = {'regex' : "2[1|3]1....11    "}
regex['REMISSAO']                       = {'regex' : "2[1|3]1....1[3|4]    "}
regex['PESL_SUS']                       = {'regex' : "2[1|3]1....2     "}
regex['PESL_OUTROS']                    = {'regex' : "2[1|3]1....3     "}
regex['PEONA_OUTROS']                   = {'regex' : "2[1|3]1....41    "}
regex['IMPOSTO SOBRE LUCRO']            = {'regex':  "61           "}
regex['TRIBUTOS DIRETOS']               = {'regex':  "32           "}
regex['RECEITA']                        = {'regex':  "3            "}
regex['DESPESA']                        = {'regex':  "4            "}
regex['TRIBUTOS DIRETOS']               = {'regex':  "32           "}
regex['RECEITA']                        = {'regex':  "3            "}
regex['DESPESA']                        = {'regex':  "4            "}
regex['OUTRAS DESPESAS OPERACIONAIS']   = {'regex':  "44           "}
regex['OUTRAS RECEITAS OPERACIONAIS']   = {'regex':  "33           "}
regex['DESPESAS FINANCEIRAS']           = {'regex':  "45           "}
regex['RECEITAS FINANCEIRAS']           = {'regex':  "35           "}
regex['PPSC']                           = {'regex':  "4419         "}
regex['APLICAÇÕES GARANTIDORAS']        = {'regex':  "1221         "}
regex['PARTICIPAÇÕES COOPERATIVAS']     = {'regex':  "132819112    "}
regex['PARTICIPAÇÕES INSTITUIÇÕES REGULADAS'] = {'regex':  "13214        "}
regex['PARTICIPAÇÕES OPS']              = {'regex':  "13211        "}
regex['CRÉDITO TRIBUTÁRIO']             = {'regex':  "1316         "}
regex['DESPESAS ANTECIPADAS']           = {'regex':  "128          "}
regex['ATIVO NÃO CIRCULANTE INTANGÍVEL']= {'regex':  "1341         "}
regex['DESPESA DE COMERCIALIZAÇÃO DIFERIDA']   = {'regex':  "1[2|3]51         "}



In [ ]:
PASTA_DEMONSTRACOES_POR_ANO = PARAMETROS_GERAIS['pasta_raiz_dados']+PARAMETROS['pasta_arquivos_contenados']

In [ ]:
lista_arquivos = [i for i in glob.glob(PASTA_DEMONSTRACOES_POR_ANO+"/*")]

In [ ]:
lista_arquivos

In [ ]:
def ordenar_dataset(data):
    return data.sort_values(by=["REG_ANS",'ANO','TRIMESTRE',"CD_CONTA_CONTABIL"])

In [ ]:
def ajustar_linhas(data):
    data['CD_CONTA_CONTABIL'] = data['CD_CONTA_CONTABIL'].str.ljust(width=13, fillchar=' ')
    data['VL_SALDO_FINAL'] = data['VL_SALDO_FINAL'].str.replace(pat=',', repl='.').astype(float)
    data = ordenar_dataset(data)
    return data

In [ ]:
def process(item,data,_param_conta):
    valor_atual = item['VL_SALDO_FINAL']
    item[_nome_conta] = str(float("{0:.2f}".format((valor_atual))))
    return item    

In [ ]:
def ajustar_contas(_nome_conta, _param_conta, data, lista_codigos):
    _ano = data['ANO'].unique().tolist()[0]
    _regex = _param_conta['regex']
    if _ano in _param_conta:
        _regex = _param_conta[_ano]
    print(_ano,_nome_conta,"-->"+_regex+"<--")
    r = re.compile(_regex)
    lista_match_regex = list(filter(r.match, lista_codigos)) 
    df_contas = data[data['CD_CONTA_CONTABIL'].isin(lista_match_regex)]
#     print(df_contas)
    df_contas.insert(loc=0, column='pos', value=np.arange(len(df_contas)))
    df_contas = df_contas.apply(lambda x : process(x,df_contas,_param_conta),axis=1)
    df_contas.drop(columns=['pos'],inplace=True)
    return df_contas

In [ ]:
PASTA_DEMONSTRACOES_CONTAS = PARAMETROS_GERAIS['pasta_raiz_dados']+os.path.sep+"DEMONSTRACOES_CONTAS"
PASTA_DEMONSTRACOES_CONTAS_SUMARIZADAS = PARAMETROS_GERAIS['pasta_raiz_dados']+os.path.sep+"DEMONSTRACOES_CONTAS_SUMARIZADAS"
if (not(os.path.exists(PASTA_DEMONSTRACOES_CONTAS))):
    os.mkdir(PASTA_DEMONSTRACOES_CONTAS)
if (not(os.path.exists(PASTA_DEMONSTRACOES_CONTAS_SUMARIZADAS))):
    os.mkdir(PASTA_DEMONSTRACOES_CONTAS_SUMARIZADAS)

In [ ]:
for _arquivo in lista_arquivos:
    ano_arquivo  = _arquivo.split(os.path.sep)[-1].split("_")[0]
    data = pd.read_csv(_arquivo, dtype=str,encoding="ISO-8859-1")
    print(_arquivo)
    data = data.set_index("CHAVE")
    data = ajustar_linhas(data)
    lista_codigos_conta_contabil = data['CD_CONTA_CONTABIL'].unique().tolist()    
    for _nome_conta, _param_conta in regex.items():
        data.insert(loc=3, column=_nome_conta, value=0.0)
        df_contas = ajustar_contas(_nome_conta,_param_conta,data,lista_codigos_conta_contabil)
        data.drop(df_contas.index,inplace=True)
        data = pd.concat([data,df_contas])
    nome_arquivo = _arquivo[_arquivo.rfind(os.path.sep)+1:]
    print(nome_arquivo)
    data = ordenar_dataset(data)
    data.to_csv(PASTA_DEMONSTRACOES_CONTAS+os.path.sep+ano_arquivo+".csv",encoding="ISO-8859-1")

### Salvando o arquivo Versão 2

In [ ]:
lista_arquivos = [i for i in glob.glob(PASTA_DEMONSTRACOES_CONTAS+os.path.sep+"*")]

In [ ]:
lista_arquivos

In [ ]:
def converter_colunas_regex_para_float(df):
    for coluna in list(regex.keys()):
        df[coluna] = df[coluna].astype(float)

In [ ]:
for arquivo in lista_arquivos:
    nome_arquivo = arquivo.split(os.path.sep)[-1]
    print(nome_arquivo)
    df_temp = pd.read_csv(arquivo,dtype=str,encoding="ISO-8859-1")
    converter_colunas_regex_para_float(df_temp)
#     df_temp = df_temp[colunas_dataset]
    print(df_temp.columns)
    df_temp = df_temp.groupby(["REG_ANS","ANO","TRIMESTRE","PERIODO"],as_index=False).sum()
    df_temp.to_csv(PASTA_DEMONSTRACOES_CONTAS_SUMARIZADAS+os.path.sep+nome_arquivo,index=False,encoding="ISO-8859-1")
    print("Salvo")

In [ ]:
lista_arquivos = [i for i in glob.glob(PASTA_DEMONSTRACOES_CONTAS_SUMARIZADAS+os.path.sep+"*")]

In [ ]:
lista_arquivos

In [ ]:
df_demonstracoes_concatenadas2 = pd.concat([pd.read_csv(f,dtype=str,encoding="ISO-8859-1") for f in lista_arquivos],sort=False)

In [ ]:
df_demonstracoes_concatenadas2.to_csv(PARAMETROS_GERAIS['pasta_raiz_dados']+PARAMETROS_GERAIS['arquivo_final_todas_demonstracoes'],encoding="ISO-8859-1",index=False)

In [ ]:
df_demonstracoes_concatenadas2.head()

In [ ]:
PARAMETROS_GERAIS['pasta_raiz_dados']+PARAMETROS_GERAIS['arquivo_final_todas_demonstracoes']